In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import pickle
import numpy as np
from evaluation_scripts.base import get_pathes
from evaluation_scripts.well_ident import build_dictionaries

experiments = ["20220412", "20220127", "20210417"]

## transfer times
- transferduration is the duration between treatment_start of a plate.

In [2]:
def get_transfer_times(times):
    treatment_start = times[times.event == "treatment_start"]
    T =  treatment_start.transfer.unique()
    T.sort()
    durations = []
    for plate in treatment_start.plate.unique():
        for transfer in T[:-1]:
            df_0 = treatment_start[(treatment_start.plate == plate) & (treatment_start.transfer == transfer) ]
            df_1 = treatment_start[(treatment_start.plate == plate) & (treatment_start.transfer == transfer+1) ]
            row = {"t_start" : df_0["time"].values[0], "t_start_next" : df_1["time"].values[0]}
            durations.append(row)

    durations = pd.DataFrame(durations)

    # Convert time columns to datetime objects
    durations["t_start"] = pd.to_datetime(durations["t_start"])
    durations["t_start_next"] = pd.to_datetime(durations["t_start_next"])

    # Calculate the duration
    durations["duration"] = (durations["t_start_next"] - durations["t_start"]).dt.total_seconds()/3600
    return durations

In [5]:
def get_incubation_times(times):
    T =  times.transfer.unique()
    T.sort()
    durations = pd.DataFrame()
    for t in T:
        df_t = times[times.transfer == t]
        if ((df_t.event == "incubation_end").sum() == 6) & ((df_t.event == "incubation_start").sum() == 6):
            cols = ["time", "plate", "transfer"]
            duration_t = df_t.loc[df_t.event == "incubation_end", cols]
            duration_t["incubation_start"] = df_t.loc[df_t.event == "incubation_start","time"].values
            duration_t = duration_t.rename(columns = {"time":"incubation_end"})
            durations = pd.concat([durations, duration_t])
            
    # Convert time columns to datetime objects
    durations["incubation_start"] = pd.to_datetime(durations["incubation_start"])
    durations["incubation_end"] = pd.to_datetime(durations["incubation_end"])

    # Calculate the duration
    durations["duration"] = (durations["incubation_end"] - durations["incubation_start"]).dt.total_seconds()/3600
    return durations


In [15]:
def print_results(df):
    print("Mean", df.duration.mean(), "pi:10%", df.duration.quantile(0.1), "pi:90%", df.duration.quantile(0.9))

In [16]:
transfer_durations = pd.DataFrame()
incubation_durations = pd.DataFrame()
for exp in experiments:
    print("\n", exp)
    
    ### LOAD
    jupyter_path, base_path, path, analysis_path, os = get_pathes(exp+"_output")
    times = pd.read_csv(os.path.join(path, "times.csv"))
    transfer_durations = pd.concat([transfer_durations, get_transfer_times(times)])
    incubation_durations = pd.concat([incubation_durations, get_incubation_times(times)])


 20220412

 20220127

 20210417


In [17]:
print_results(transfer_durations)

Mean 29.504547784391537 pi:10% 21.96527777777778 pi:90% 47.175694444444446


In [18]:
print_results(incubation_durations)

Mean 27.27730631141345 pi:10% 19.67822222222222 pi:90% 39.554027777777776
